In [25]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
import mne

Konfiguracja modelu

In [26]:
train_files=['p10_Record2.edf', 'p11_Record1.edf', 'p11_Record2.edf', 'p11_Record3.edf', 'p11_Record4.edf', 'p12_Record1.edf', 'p12_Record2.edf', 'p12_Record3.edf', 'p13_Record1.edf', 'p13_Record2.edf', 'p13_Record3.edf', 'p13_Record4.edf', 'p14_Record1.edf', 'p14_Record2.edf', 'p14_Record3.edf', 'p15_Record1.edf', 'p15_Record2.edf', 'p15_Record3.edf', 'p15_Record4.edf']
test_file= 'p10_Record1.edf'
frame_size=1000
units_of_LSTM=10
number_of_epochs=5

Model sequential jest potokiem.
Definiowanie modelu:
    LSTM units: ile równolegle ma być komórek typu LSTM 
    Return_sequences: czy lstm ma zwracać sekwencje
    Input_shape: (ilość elementów w jednej ramce,liczba cech)
    Dense: warstwa najprostszych modeli neuronów by złożyć nam neurony lstm w jedno
Kompilowanie 
    loss: Funkcja oceny

In [27]:
x=[]
y=[]
model = Sequential([
    LSTM(units=units_of_LSTM, input_shape=(frame_size, 19),return_sequences=True),
    Dense(units=19)
])
model.compile(optimizer='adam', loss='mean_squared_error')

Odczytanie pliku.
Wybranie dziewietnastu kanałów.
Obliczenie rozmiaru ramki.
Przyporządkowanie ramce kolejnej ramki.
Usunięcie ostatniej ramki i ramki jej przyporządkowanej
Trenowanie modelu:
    batch_size=rozmiar użytych próbek
    epochs= liczba epok

In [ ]:
for file in train_files:
    x=[]
    y=[]
    open_t=mne.io.read_raw_edf(file)
    raw_data=open_t.get_data()
    open_t=0
    channel=raw_data[:19]
    numbers_of_frames=int(channel[0].size/frame_size)
    for frame in range(numbers_of_frames):
        x_to_add=channel[:,frame*frame_size:frame*frame_size+frame_size]
        y_to_add=channel[:,frame*frame_size+frame_size:frame*frame_size+frame_size*2]
        x_to_add=np.array(x_to_add).T
        y_to_add=np.array(y_to_add).T
        x.append(x_to_add.tolist())
        y.append(y_to_add.tolist())
    y_to_add=[]
    x_to_add=[]
    x.pop()
    y.pop()
    x=tf.convert_to_tensor(x)
    y=tf.convert_to_tensor(y)
    model.fit(x, y, batch_size=frame_size, epochs=number_of_epochs,use_multiprocessing=True)

Odczyt i testowanie modelu

In [ ]:
train=mne.io.read_raw_edf(test_file)
train=train.get_data()
do_testowania_array=np.array(train[:19,:frame_size])

Predykcja na podstawie modelu

In [ ]:
do_testowania_array=do_testowania_array.T
wynik = model.predict([do_testowania_array.tolist()])

Rysowanie wykresu predykcji i oryginalnej wartości

In [ ]:
wynik=wynik[0]
wynik=wynik.T
for kanal in range(19):
    plt.plot(train[kanal][frame_size:frame_size*2])
    plt.plot(wynik[kanal])
    plt.show()